In [ ]:
import pandas as pd

def match_quat_accel_gyro():

    # =========================
    # CUATERNIONES
    # =========================
    df_q = pd.read_csv("quat3.csv")

    df_q = df_q.rename(columns={
        "inertial-6286.188861:estOrientQuaternion[0-0]": "q0",
        "inertial-6286.188861:estOrientQuaternion[0-1]": "q1",
        "inertial-6286.188861:estOrientQuaternion[0-2]": "q2",
        "inertial-6286.188861:estOrientQuaternion[0-3]": "q3",
        "inertial-6286.188861:estOrientQuaternion:valid": "qValid",
    })

    df_q = df_q[df_q["qValid"] == 1]

    df_q["time_comp"] = (df_q["Time"] * 1e-8).astype(int)

    # =========================
    # ACELERACIONES
    # =========================
    df_a = pd.read_csv("accel3.csv")

    df_a = df_a.rename(columns={
        "inertial-6286.188861:estLinearAccelX": "ax",
        "inertial-6286.188861:estLinearAccelX:valid": "aXvalid",
        "inertial-6286.188861:estLinearAccelY": "ay",
        "inertial-6286.188861:estLinearAccelY:valid": "aYvalid",
        "inertial-6286.188861:estLinearAccelZ": "az",
        "inertial-6286.188861:estLinearAccelZ:valid": "aZvalid",
    })

    df_a = df_a[
        (df_a["aXvalid"] == 1) &
        (df_a["aYvalid"] == 1) &
        (df_a["aZvalid"] == 1)
    ]

    df_a["time_comp"] = (df_a["Time"] * 1e-8).astype(int)

    # =========================
    # GIROSCOPIO (ANGULAR RATE)
    # =========================
    df_g = pd.read_csv("gyro3.csv", skiprows=1)  # DATA_START

    df_g = df_g.rename(columns={
        "inertial-6286.188861:estAngularRateX": "wx",
        "inertial-6286.188861:estAngularRateX:valid": "wXvalid",
        "inertial-6286.188861:estAngularRateY": "wy",
        "inertial-6286.188861:estAngularRateY:valid": "wYvalid",
        "inertial-6286.188861:estAngularRateZ": "wz",
        "inertial-6286.188861:estAngularRateZ:valid": "wZvalid",
    })

    df_g = df_g[
        (df_g["wXvalid"] == 1) &
        (df_g["wYvalid"] == 1) &
        (df_g["wZvalid"] == 1)
    ]

    df_g["time_comp"] = (df_g["Time"] * 1e-8).astype(int)

    # =========================
    # MERGES (INTERSECCIÓN)
    # =========================
    df_u = pd.merge(df_q, df_a, on="time_comp", how="inner")
    df_u = pd.merge(df_u, df_g, on="time_comp", how="inner")

    # =========================
    # LIMPIEZA FINAL
    # =========================
    df_u = df_u.rename(columns={"Time_x": "time"})
    df_u.drop(columns=[c for c in df_u.columns if "Time_" in c], inplace=True)

    df_u["dt"] = df_u["time"].diff() * 1e-9

    new_order = [
        "time",
        "q0", "q1", "q2", "q3",
        "ax", "ay", "az",
        "wx", "wy", "wz",
        "dt"
    ]

    df_u = df_u[new_order]

    return df_u


df = match_quat_accel_gyro()
df